# Internal QA sheets 

This script is designed to test the VD outputs of multiple scenarios 

Scenarios can be selected by version number and "run" name (eg kea, tui, etc)

### Notes 

We note that this will not consider the current post-processing system of "raw", "main", and "clean" data. 

This attempts to look at the actual model outputs while keeping handling as light as possible. 

Long term, the work done here might also be used for post-processing. Current post-processing has extra data handling which means it deviates from model outputs. This should be cleaned up. 

Separately, we want to create a big cool graph object for any given scenario run. This will be a separate project

### Concordance

The concordance file (containing Att/Proc/Com metadata) is key to interpreting the TIMES-NZ specific outputs.

The actual concordance data can be adjusted. It is found here:

`TIMES-NZ-OUTPUT-PROCESSING/data/input/concordance/attribute_process_commodity_concordance.csv`. 

The interactive version can be browsed here: 

`TIMES-NZ-OUTPUT-PROCESSING/data/input/concordance/times_concordance_lookup.html`

If the data is updated, the lookup file can be updated by navigating to `TIMES-NZ-OUTPUT-PROCESSING\data\input\concordance` and running `create_lookup.py`

### Steps: 

1) Define scenarios, versions 
2) Define methods for accessing relevant attributes/processes/fuels from the specified scenarios/versions
3) Run check testing for any differences: 
    - new/missing attributes/processes/fuels 
    - within matching attributes/processes/fuels, where values are different
    - output a report 
4) Output charts for visual assessments of differences
    - Input differences (not VD files! This will need to wait for a better input data structure) 
    - Emissions
      - Total
      - Sector
      - Fuel 
    - Supply
      - to assess value of this? Not sure how interesting it is 
    - End use
      - Sectors by fuel
      - ESD (tech-agnostic)
      - Technology uptake 
    - Generation
      - Generation mix
      - timeslices/peaking/peak cap
      - Required build
    - Prices
      - (not input prices, these should be covered separately)
      - Need to explore these better and ensure they make sense











## SETUP

In [2]:
# libraries 
import pandas as pd
# import numpy as np
import re
import os 
from pathlib import Path

import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
# scenario definitions 

# these should match the folder names in times_scenarios/ that we want to assess 
qa_runs = ["tui-v2_1_2", "tui-v2_1_3"]
# we could do something fancy by saying like "i want every version of tui" or whatever,
# but this is flexible to naming conventions changing in future 


In [4]:
# checking directories and data 

# TIMES_LOCATION = os.getcwd()
# go up one 
TIMES_LOCATION = os.path.dirname(os.path.dirname(os.getcwd()))
# identify data location 
TIMES_OUTPUTS_RAW = os.path.join(TIMES_LOCATION, "TIMES-NZ-GAMS", "times_scenarios")

# test that your scenarios exist 

# (to do: go down a bit and make sure the vd files exist)
available_runs = os.listdir(TIMES_OUTPUTS_RAW)
print(f"Checking data availability:")

for run in qa_runs: 
    if run not in available_runs:
        print(f"Warning: `{run}` cannot be found in the current run data. Ensure you have processed veda correctly")
    else: 
        print(f"`{run}` found :)")






Checking data availability:
`tui-v2_1_2` found :)
`tui-v2_1_3` found :)


In [5]:
# data retrival functions 


def read_vd(filepath):
    """
    Reads a VD file, using column names extracted from the file's
    header with regex, skipping non-CSV formatted header lines.

    :param filepath: Path to the VD file.
    :param scen_label: Label for the 'scen' column for rows from this file.
    """
    dimensions_pattern = re.compile(r"\*\s*Dimensions-")

    # Determine the number of rows to skip and the column names
    with open(filepath, "r", encoding="utf-8") as file:
        columns = None
        skiprows = 0
        for line in file:
            if dimensions_pattern.search(line):
                columns_line = line.split("- ")[1].strip()
                columns = columns_line.split(";")
                continue
            if line.startswith('"'):
                break
            skiprows += 1

    # Read the CSV file with the determined column names and skiprows
    vd_df = pd.read_csv(
        filepath, skiprows=skiprows, names=columns, header=None, low_memory=False
    )
    return vd_df


def get_attribute_df_single_run(attribute, run_name): 
    """
    Reads a TIMES vd file and returns a dataframe filtered to a specific attribute 
    
    Parameters:
    attribute (str): Attribute from a TIMES vd file to filter on 
    run_name (str): The name of the run this file is from. This must exist in the specified df
    
    Returns:
    Dataframe - untouched vd output but on a specific attribute and with a run label 
    """

    filepath = os.path.join(TIMES_OUTPUTS_RAW, run_name, f"{run_name}.vd")
    vd = read_vd(filepath)
    vd = vd[vd["Attribute"] == attribute]
    vd["Scenario"] = run_name
    return vd
    
def add_concordance_to_vd(df):
    """
    Takes a TIMES VD file with Attribute/Process/Commodity data, and adds the local concordance file to it 
    Hardcodes the concordance file location, which must exist
    
    Parameters:
    df (str): the pandas dataframe we're adding the concordance to   

    Returns:
    a df with the new variables attached
    """

    concordance_filepath = os.path.join(TIMES_LOCATION,
                                        "TIMES-NZ-OUTPUT-PROCESSING/data/input/concordance/",
                                        "attribute_process_commodity_concordance.csv")
    
    concordance_data = pd.read_csv(concordance_filepath)

    df = df.merge(concordance_data,
                  how = "left",
                  on = ["Attribute", "Process", "Commodity"])
    return df

    

def get_veda_data_no_concordance(attribute, runs = qa_runs):
    """
    Combines previous functions to get the same attribute data for all relevant runs
    then attach the concordance file for easy perusal. 
    
    Parameters:
    attribute (str): the VD attribute this pulls
    runs (array): an array of run names stored as strings. Defaults to the `qa_runs` defined earlier 

    Returns:
    a df 
    """
    df = pd.DataFrame()

    for run in runs: 
        vd_df = get_attribute_df_single_run(attribute, run)
        df = pd.concat([df, vd_df])    

    return df


def get_veda_data(attribute, runs = qa_runs):
    
    df = get_veda_data_no_concordance(attribute, runs)
    df = add_concordance_to_vd(df)

    return df 



    

In [6]:
# get data 
fuel_use_df = get_veda_data("VAR_FIn")
output_df = get_veda_data("VAR_FOut")
cap_df = get_veda_data("VAR_Cap")


In [7]:
# define aggregation categories
# to this you can group by other stuff if needed like process/commopdity/sector or whatever but this will form the baseline? 
standard_group_categories = [
    "Attribute",   
    "Period",
    "Scenario",
    "Unit"
]

## Automated Testing

In [8]:
# Testing functions 

# for each attribute of interest (generalisable)
# we want to take both the relevant tables and: 

    # a) run minus tests in both directions (covers all differences)
    # b) identify all rows where everything matches except the data (ie the PV value). THis is some merge stuff probably? 
    # c) identify all rows where there are category differences agnostic to data. This is basically a minus test where we remove the PV before checking anything.


# When doing structure minus testing, we have to be quite careful. TIMES will output 0s as implicit missing values. This means if a value is 0 in one run, and over 0 in another,
# This will flag as a structure issue 

# So for the main minus testing we remove all time periods, time slices, and vintage categories, and just focus on the att/proc/comm combinations 

# An alternative approach would be to fill all the missings with explicit zeroes, but it would be best to do that in categories that align across both categories so we can identify data differences
# within matching categories 

# pd.reset_option('display.max_rows')
test_attribute = "VAR_FIn"


def get_data_structure(attribute):
    # the point of this is to return the dataframe with no numbers in it so we can test structure changes 
    # while we are here we are going 
    df = get_veda_data_no_concordance(attribute)
    # remove PV and ensure grain holds 


    current_count = len(df)
    
    df_no_pv = df.drop("PV", axis = 1).drop_duplicates()
    new_count = len(df_no_pv) 

    if current_count != new_count:
        print(f"Error: '{attribute}' data has {current_count} rows")
        print(f"However, it has {new_count} rows after removing PV and collapsing")
        print(f"Mismatched counts imply rows are not uniquely defined in Veda output!! Help!!")

    return df_no_pv

def remove_time_periods(df):

    df = df.drop("Period", axis = 1).drop_duplicates()
    df = df.drop("TimeSlice", axis = 1).drop_duplicates()
    df = df.drop("Vintage", axis = 1).drop_duplicates()

    return df

def minus_test_structure(attribute, run_a, run_b, max_rows = 100): 

    df = get_data_structure(attribute) 
    # drop times to remove noise 
    df = remove_time_periods(df) 

    # get separate tables     
    df_a = df[df["Scenario"] == run_a].drop("Scenario", axis = 1)
    df_b = df[df["Scenario"] == run_b].drop("Scenario", axis = 1)  

    # minus 
    a_minus_b = (    
        df_a
        .merge(df_b, how='left', indicator=True)
        .query('_merge == "left_only"')
        .drop('_merge', axis=1)
        )
    
    # assess result 
    if len(a_minus_b) > 0: 
       print(f"The following structures are defined in '{run_a}' but not '{run_b}'")
       a_minus_b = add_concordance_to_vd(a_minus_b)
       pd.set_option('display.max_rows', max_rows)
       return a_minus_b
    else: 
        print(f"All CAP, ATT, and PROC combinations in '{run_b}' are also found in '{run_a}'")    

    # a - b 





### Minus testing 

In [9]:
# VAR_FIN A MINUS B
minus_test_structure("VAR_FIn", qa_runs[0], qa_runs[1])

The following structures are defined in 'tui-v2_1_2' but not 'tui-v2_1_3'


,Attribute,Commodity,Process,Region,UserConstraint,Sector,Subsector,Technology,Fuel,Enduse,Unit,Parameters,FuelGroup
0,VAR_FIn,COA,IIS-FDSTCK-COA-_,NI,-,Industry,Iron/Steel,Feedstock,Coal,Feedstock,PJ,Feedstock,Fossil Fuels
1,VAR_FIn,NGA,MTHOL-FDSTCK-NGA-FDSTCK,NI,-,Industry,Methanol,Feedstock,Natural Gas,Feedstock,PJ,Feedstock,Fossil Fuels
2,VAR_FIn,INDNGA,MTHOL-PH_REFRM-NGA-REFRM,NI,-,Industry,Methanol,Reformer,Natural Gas,Process Heat Reformer,PJ,Fuel Consumption,Fossil Fuels


In [10]:
# VAR_FIN B MINUS A
minus_test_structure("VAR_FIn", qa_runs[1], qa_runs[0])

The following structures are defined in 'tui-v2_1_3' but not 'tui-v2_1_2'


,Attribute,Commodity,Process,Region,UserConstraint,Sector,Subsector,Technology,Fuel,Enduse,Unit,Parameters,FuelGroup
0,VAR_FIn,INDFOL,OTH-FOL-FOL-Tech15,NI,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,VAR_FIn,COA,TU_COA_SI_NI_01,SI,-,NaN,NaN,NaN,Coal,NaN,PJ,NaN,Fossil Fuels
2,VAR_FIn,INDFOL,OTH-FOL-FOL-Tech15,SI,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,VAR_FIn,INDELC,PLPPPR-AIR-ELC-CMPR15,SI,-,Industry,Pulp and Paper Manufacturing,Compressor,Electricity,Compressed Air,PJ,Fuel Consumption,Electricity


In [11]:
# VAR_FOUT A MINUS B
minus_test_structure("VAR_FOut", qa_runs[0], qa_runs[1])

The following structures are defined in 'tui-v2_1_2' but not 'tui-v2_1_3'


,Attribute,Commodity,Process,Region,UserConstraint,Sector,Subsector,Technology,Fuel,Enduse,Unit,Parameters,FuelGroup
0,VAR_FOut,IIS-FDSTCK,IIS-FDSTCK-COA-_,NI,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,VAR_FOut,MTHOL-FDSTCK,MTHOL-FDSTCK-NGA-FDSTCK,NI,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,VAR_FOut,INDCO2,MTHOL-PH_REFRM-NGA-REFRM,NI,-,Industry,Methanol,Reformer,Natural Gas,Process Heat Reformer,kt CO2,Emissions,Fossil Fuels
3,VAR_FOut,MTHOL-PH_REFRM,MTHOL-PH_REFRM-NGA-REFRM,NI,-,Industry,Methanol,Reformer,Natural Gas,Process Heat Reformer,PJ,End Use Demand,Fossil Fuels


In [12]:
# VAR_FOUT B MINUS A
minus_test_structure("VAR_FOut", qa_runs[1], qa_runs[0])

The following structures are defined in 'tui-v2_1_3' but not 'tui-v2_1_2'


,Attribute,Commodity,Process,Region,UserConstraint,Sector,Subsector,Technology,Fuel,Enduse,Unit,Parameters,FuelGroup
0,VAR_FOut,COA,TU_COA_SI_NI_01,NI,-,NaN,NaN,NaN,Coal,NaN,PJ,NaN,Fossil Fuels
1,VAR_FOut,INDCO2,OTH-FOL-FOL-Tech15,NI,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,VAR_FOut,OTH-FOL,OTH-FOL-FOL-Tech15,NI,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,VAR_FOut,INDCO2,OTH-FOL-FOL-Tech15,SI,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,VAR_FOut,OTH-FOL,OTH-FOL-FOL-Tech15,SI,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,VAR_FOut,PLPPPR-AIR,PLPPPR-AIR-ELC-CMPR15,SI,-,Industry,Pulp and Paper Manufacturing,Compressor,Electricity,Compressed Air,PJ,End Use Demand,Electricity


In [34]:
def get_minus_test_df(df_a, df_b):

    a_minus_b = (    
        df_a
        .merge(df_b, how='left', indicator=True)
        .query('_merge == "left_only"')
        .drop('_merge', axis=1)
        )
    return(a_minus_b)

def check_minus_test(df, run_a, run_b, variable, display = True):    
    
    df_a = df[df["Scenario"] == run_a].drop("Scenario", axis = 1)
    df_b = df[df["Scenario"] == run_b].drop("Scenario", axis = 1)

    a_minus_b = get_minus_test_df(df_a,df_b)

    if len(a_minus_b) == 0:
        print(f"All instances of '{variable}' in '{run_a}' are also in '{run_b}'")

    else: 
        failed_categories = a_minus_b[variable]      

        if display:     
            print(f"The following instances of '{variable}' in '{run_a}' are not in `{run_b}`:")
            for category in failed_categories:
                print(f" - {category}")

        else: 
            return failed_categories
        

def check_category_mismatch(attribute, variable, run_a, run_b):    

    print(f"{attribute}: checking {variable} matches")
    # get the atty data 
    df = get_veda_data_no_concordance(attribute)
    # identify unique values of variable
    df = df[[variable, "Scenario"]].drop_duplicates()

    check_minus_test(df, run_a, run_b, variable)
    check_minus_test(df, run_b, run_a, variable)
    print()
    
def check_all_category_mismatches(run_a, run_b):

    attributes_to_check = ["VAR_FIn", "VAR_FOut", "VAR_CAP"]
    variables_to_check = ["Process", "Commodity"]


    for attribute in attributes_to_check:
        for variable in variables_to_check:
            check_category_mismatch(attribute, variable, run_a, run_b)





In [ ]:
run_a = qa_runs[0]
run_b = qa_runs[1]

check_all_category_mismatches(run_a, run_b)



#check_missing_category("VAR_FIn", "Commodity")

In [ ]:
## CHECK DATA CHANGES 



"""
METHOD


We want to test any data changes given the grain of attribute, process, commodity, period, and region.
Put another way, we want to hold those things constant between tables (so trim both to match) and then assess which ones have had values change

We will assess only if numbers change within these grains 
We won't capture if numbers changed because the structures of these changed (eg a process was renamed),
because that testing is done elsewhere.

This means the aggregated output of these might be the same at a Period level!
However we will capture if eg; some demand has shifted in its timeslice.
It is better to assess that here, rather than chalking it up to a category change in our category change testing.
Timeslice or vintage changes can be a bit overwhelming/noisy

"""



    
    

run_a = qa_runs[0]
run_b = qa_runs[1]

constant_variables = ["Attribute", "Commodity", "Process", "Period", "Region"]


# get all unique combinations for each scenario

only_variables = (output_df.drop_duplicates(subset = constant_variables + ["Scenario"]))     

variables_a = only_variables[only_variables["Scenario"] == run_a].drop("Scenario", axis = 1)
variables_b = only_variables[only_variables["Scenario"] == run_b].drop("Scenario", axis = 1)

matching_variables = pd.merge(
    variables_a, variables_b, 
    on = constant_variables,
    suffixes = ("_a", "_b")
)




# first, aggregate the main table by the constant variables and scenario 

agg_table = (output_df
             .groupby(constant_variables + ["Scenario"])
             .sum("PV").reset_index()
)

# retain all other variables in pivot
index_cols = [col for col in agg_table.columns if col not in ["PV", "Scenario"]]

delta_test = agg_table.pivot(    
    index = index_cols,
    columns = "Scenario",
    values = "PV"
).fillna(0).reset_index()


delta_test["Delta"] = delta_test[run_a] - delta_test[run_b]

# tolerance within 0.1% change
# i thought i would need to mess around with the way it handles 0s but it seems to work fine 
delta_test["Delta_Proc"] = delta_test[run_b] / delta_test[run_a] - 1
delta_test =  delta_test[(abs(delta_test["Delta_Proc"]) > 0.001 )] 


# add concordance back 
delta_test = add_concordance_to_vd(delta_test)

#delta_test = delta_test.groupby(["Attribute", "Commodity", "Process"]).size().reset_index(name = "diff_count")

#$delta_test = add_concordance_to_vd(delta_test)

pd.set_option('display.max_rows', 10              )


changed_grain = delta_test.groupby(["Attribute", "Process", "Commodity"]).size().reset_index(name = "Count").drop("Count", axis = 1 )


delta_data = output_df.merge(changed_grain)

df = delta_data

sector_options = delta_data["Sector"].unique()
parameter_options = delta_data["Parameters"].unique()


sector_options

df["Parameters"].unique()
df.columns

# Replace NaN with "Missing" in those columns
string_columns = df.select_dtypes(include=['object']).columns
string_columns = [col for col in string_columns if col != 'PV']
df[string_columns] = df[string_columns].fillna('Missing')

sector_options = df["Sector"].unique().tolist()
parameter_options = df["Parameters"].unique().tolist()

df

In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# Define the drill-down hierarchy
HIERARCHY = ['Fuel', 'Subsector', 'Technology', 'TimeSlice']

# Initialize the Dash app
app = dash.Dash(__name__)

# Initialize the layout with a store component
app.layout = html.Div([
    # Store for keeping track of the current view state
    dcc.Store(id='view-state', data={
        'level': 0,  # Index into HIERARCHY
        'selections': {}  # Will store {field: value} pairs as we drill down
    }),
    
    html.H1('Energy System Analysis Dashboard', 
            style={'textAlign': 'center', 'marginBottom': '20px'}),
            
    # Header area for current filters
    html.Div(id='current-filters', style={'marginBottom': '20px', 'padding': '10px', 'backgroundColor': '#f8f9fa'}),
    
    # Current view title
    html.H2(id='view-title', style={'textAlign': 'center', 'marginBottom': '20px'}),
    
    # Container for filters and navigation
    html.Div([
        # Parameter dropdown
        html.Div([
            html.Label('Select Parameter:'),
            dcc.Dropdown(
                id='parameter-dropdown',
                options=[{'label': param, 'value': param} for param in parameter_options],
                value=parameter_options[0]
            )
        ], id='param-dropdown-container', style={'width': '30%', 'display': 'inline-block'}),
        
        # Back button (initially hidden)
        html.Button(
            'Back',
            id='back-button',
            style={'display': 'none', 'marginLeft': '20px'},
        )
    ], style={'marginBottom': '20px'}),
    
    # Main graph
    html.Div([
        dcc.Graph(id='faceted-chart', style={'height': '800px'}),
    ])
])

# Callback to handle clicks and view state
@app.callback(
    [Output('view-state', 'data'),
     Output('back-button', 'style'),
     Output('param-dropdown-container', 'style'),
     Output('current-filters', 'children'),
     Output('view-title', 'children')],
    [Input('faceted-chart', 'clickAnnotationData'),
     Input('back-button', 'n_clicks')],
    [State('view-state', 'data'),
     State('parameter-dropdown', 'value')]
)
def update_view_state(annotation_click, back_clicks, current_state, selected_parameter):
    ctx = dash.callback_context
    if not ctx.triggered:
        # Generate initial filter display
        filter_display = [html.P(f"Parameter: {selected_parameter}", style={'fontWeight': 'bold', 'marginBottom': '5px'})]
        view_title = HIERARCHY[current_state['level']] + 's'
    
        return current_state, {'display': 'none'}, {'width': '30%', 'display': 'inline-block'}, filter_display, view_title
    
    trigger = ctx.triggered[0]['prop_id'].split('.')[0]
    
    if trigger == 'back-button':
        # Go back one level
        new_level = max(0, current_state['level'] - 1)
        new_selections = dict(list(current_state['selections'].items())[:new_level])
        
        # Show dropdown only at top level
        dropdown_style = {'width': '30%', 'display': 'inline-block'} if new_level == 0 else {'display': 'none'}
        # Show back button except at top level
        button_style = {'display': 'none'} if new_level == 0 else {'display': 'inline-block', 'marginLeft': '20px'}
        
        # Generate filter display
        filter_display = [html.P(f"Parameter: {selected_parameter}", style={'fontWeight': 'bold', 'marginBottom': '5px'})]
        for field in HIERARCHY[:new_level]:
            if field in new_selections:
                filter_display.append(html.P(f"{field}: {new_selections[field]}", 
                                          style={'fontWeight': 'bold', 'marginBottom': '5px'}))
        
        # Generate view title
        view_title = 'Select ' + HIERARCHY[new_level] # could also pluralise but "technologies" is annoying
        
        return {
            'level': new_level,
            'selections': new_selections
        }, button_style, dropdown_style, filter_display, view_title
    
    if trigger == 'faceted-chart' and annotation_click:
        # Clean the selected value (remove arrows and whitespace)
        selected_value = annotation_click['annotation']['text'].replace('⯆', '').strip()
        
        # Get the current field we're looking at
        current_field = HIERARCHY[current_state['level']]
        
        # Update selections with the new value
        new_selections = dict(current_state['selections'])
        new_selections[current_field] = selected_value
        
        # Move to next level if not at end
        new_level = min(len(HIERARCHY) - 1, current_state['level'] + 1)
        
        # Generate filter display
        filter_display = [html.P(f"Parameter: {selected_parameter}", style={'fontWeight': 'bold', 'marginBottom': '5px'})]
        for field in HIERARCHY[:new_level + 1]:
            if field in new_selections:
                filter_display.append(html.P(f"{field}: {new_selections[field]}", 
                                          style={'fontWeight': 'bold', 'marginBottom': '5px'}))
        
        # Generate view title
        view_title = HIERARCHY[new_level] + 's'  # Pluralize
        
        return {
            'level': new_level,
            'selections': new_selections
        }, {'display': 'inline-block', 'marginLeft': '20px'}, {'display': 'none'}, filter_display, view_title
    
    return current_state, {'display': 'none'}, {'width': '30%', 'display': 'inline-block'}


# Callback to update the graph
@app.callback(
    Output('faceted-chart', 'figure'),
    [Input('parameter-dropdown', 'value'),
     Input('view-state', 'data')]
)
def update_graph(selected_parameter, view_state):
    # Start with parameter filter
    filtered_df = df[df['Parameters'] == selected_parameter]
    
    # Apply all accumulated filters
    for field, value in view_state['selections'].items():
        filtered_df = filtered_df[filtered_df[field] == value]
    
    # Get current faceting field
    facet_column = HIERARCHY[view_state['level']]
    
    # Aggregate data
    agg_df = filtered_df.groupby(['Period', facet_column, 'Scenario'])['PV'].sum().reset_index()
    
    # Get available facet values
    facet_values = sorted(agg_df[facet_column].unique())
    
    # Handle empty dataset case
    if len(facet_values) == 0:
        fig = go.Figure()
        fig.add_annotation(
            text='No data available for the selected filters',
            xref="paper",
            yref="paper",
            x=0.5,
            y=0.5,
            showarrow=False,
            font=dict(size=14)
        )
        return fig

    # Calculate number of rows and columns for subplots
    n_facets = len(facet_values)
    n_cols = min(6, max(1, n_facets))
    n_rows = max(1, (n_facets + n_cols - 1) // n_cols)
    
    # Create subplot figure
    fig = make_subplots(
        rows=n_rows, cols=n_cols
        #vertical_spacing=0.3,
        #horizontal_spacing=0.05
    )
    
    # Color mapping for scenarios
    color_map = {run_a: 'red', run_b: 'blue'}
    
    # Add traces for each facet value
    for idx, facet_value in enumerate(facet_values):
        row = idx // n_cols + 1
        col = idx % n_cols + 1
        
        facet_data = agg_df[agg_df[facet_column] == facet_value]
        
        # Add title annotation with different styling based on whether we can drill deeper
        can_drill_deeper = view_state['level'] < len(HIERARCHY) - 1
        if can_drill_deeper:
            fig.add_annotation(
                text='⯆ ' + facet_value + ' ⯆',
                xref="x domain",
                yref="y domain",
                x=0.5,
                y=1.1,
                showarrow=False,
                xanchor="center",
                yanchor="bottom",
                row=row,
                col=col,
                font=dict(size=12, color='#0066cc'),
                bgcolor='rgba(240, 248, 255, 0.8)',
                bordercolor='#0066cc',
                borderwidth=1,
                borderpad=4,
                hovertext='Click to drill down',
            )
        else:
            fig.add_annotation(
                text=facet_value,
                xref="x domain",
                yref="y domain",
                x=0.5,
                y=1.1,
                showarrow=False,
                xanchor="center",
                yanchor="bottom",
                row=row,
                col=col,
                font=dict(size=12)
            )
        
        for scenario in [run_a, run_b]:
            scenario_data = facet_data[facet_data['Scenario'] == scenario]
            
            fig.add_trace(
                go.Bar(
                    x=scenario_data['Period'],
                    y=scenario_data['PV'],
                    name=scenario,
                    marker_color=color_map[scenario],
                    showlegend=True if idx == 0 else False,
                ),
                row=row, col=col
            )
    
    # Simple title for the graph
    title_text = f"{facet_column} Comparison"
    
    fig.update_layout(
        height=300 * n_rows,
        title_text=title_text,
        barmode='group',
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    
    # Update axes labels
    fig.update_xaxes(title_text="Period")
    fig.update_yaxes(title_text="Value")
    
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
# Execute tests


# first, can only be done with 2 input runs. 

# we also define which attributes we want to check:

attributes_to_test = ["VAR_FIn", "VAR_FOut", "VAR_Cap"]

if len(qa_runs) == 2:
    print("Running automated difference testing")
else:
    if len(qa_runs < 2): 
        wrong_runs_message = "Warning: you have selected less than 2 runs."
    if len(qa_runs > 2):
        wrong_runs_message = "Warning: you have selected more than 2 runs."
    print(f"{wrong_runs_message} Skipping automated testing")




## EMISSIONS

In [ ]:
# TOTAL EMISSIONS 
totcap_data = output_df[output_df["Commodity"] == "TOTCO2"]
print(f"Warning: making manual adjustments to units for TOTCO2. This should really be done in the concordance file if these are needed")
totcap_data.loc[:,"Unit"] = "ktco2e"

# all emissions by scenario, year 
totcap_data = (
    totcap_data 
    .groupby(standard_group_categories)
    .sum(["PV"])
    .reset_index()
)
# convert to MT 
totcap_data["PV"] /= 1000
totcap_data["Unit"] = "MT CO2"

sns.relplot(
    data = totcap_data, 
    x = "Period", y = "PV", hue = "Scenario",
    kind = "line",
)
plt.title("Total Emissions")
plt.xlabel("Year")
plt.ylabel("Mt CO2") # not co2e?? 
plt.show()


In [ ]:
# POSITIVE EMISSIONS

totcap_data = output_df[output_df["Commodity"] == "TOTCO2"]
print(f"Warning: making manual adjustments to units for TOTCO2. This should really be done in the concordance file if these are needed")
totcap_data.loc[:,"Unit"] = "ktco2e"

totcap_data_positive = totcap_data[totcap_data["PV"] > 0]

# all emissions by scenario, year 
totcap_data_positive = (
    totcap_data_positive 
    .groupby(standard_group_categories)
    .sum(["PV"])
    .reset_index()
)
# convert to MT 
totcap_data_positive["PV"] /= 1000
totcap_data_positive["Unit"] = "MT CO2"

sns.relplot(
    data = totcap_data_positive, 
    x = "Period", y = "PV", hue = "Scenario",
    kind = "line"    
)
plt.title("Positive Emissions")
plt.xlabel("Year")
plt.ylabel("Mt CO2") # not co2e?? 
plt.show()

In [ ]:
# NEGATIVE EMISSIONS

totcap_data = output_df[output_df["Commodity"] == "TOTCO2"]
print(f"Warning: making manual adjustments to units for TOTCO2. This should really be done in the concordance file if these are needed")
totcap_data.loc[:,"Unit"] = "ktco2e"

totcap_data_negative = totcap_data[totcap_data["PV"] < 0]

# all emissions by scenario, year 
totcap_data_negative = (
    totcap_data_negative 
    .groupby(standard_group_categories)
    .sum(["PV"])
    .reset_index()
)
# convert to MT 
totcap_data_negative["PV"] /= 1000
totcap_data_negative["Unit"] = "MT CO2"

sns.relplot(
    data = totcap_data_negative, 
    x = "Period", y = "PV", hue = "Scenario",
    kind = "line"    
)
plt.title("NEGATIVE Emissions")
plt.xlabel("Year")
plt.ylabel("Mt CO2") # not co2e?? 
plt.show()

In [ ]:
# EMISSIONS BY SECTOR


detailed_emissions = output_df[output_df["Parameters"] == "Emissions"]
# hello can we please check that the emissions parameter covers everything 
# this should add to totco2
# the original gets everything where the commodity has a co2 in it but excludes all the totco2 


detailed_emissions = (
    detailed_emissions
    .groupby(standard_group_categories + ["Sector"])
    .sum("PV")
    .reset_index()
)
detailed_emissions["PV"] /= 1000

g = sns.relplot(
    data = detailed_emissions, 
    x = "Period", y = "PV", hue = "Scenario",
    kind = "line",
    col = "Sector", 
    col_wrap = 5,
    facet_kws={'sharey': False}
)
g.figure.suptitle("Emissions by Sector", y = 1.02, size = 20)
g.set_ylabels("MT CO2")
g.set_xlabels("Year")

g.set_titles(col_template="{col_name}", size = 14)
g




In [ ]:
# EMISSIONS BY FUEL


detailed_emissions = output_df[output_df["Parameters"] == "Emissions"]
# hello can we please check that the emissions parameter covers everything 
# this should add to totco2
# the original gets everything where the commodity has a co2 in it but excludes all the totco2 


detailed_emissions = (
    detailed_emissions
    .groupby(standard_group_categories + ["Fuel"])
    .sum("PV")
    .reset_index()
)
detailed_emissions["PV"] /= 1000

g = sns.relplot(
    data = detailed_emissions, 
    x = "Period", y = "PV", hue = "Scenario",
    kind = "line",
    col = "Fuel", 
    col_wrap = 5,
    facet_kws={'sharey': False}
)
g.figure.suptitle("Emissions by Fuel", y = 1.02, size = 20)
g.set_ylabels("MT CO2")
g.set_xlabels("Year")

g.set_titles(col_template="{col_name}", size = 14)
g


## ENERGY DEMAND

In [ ]:
# ELECTRICITY DEMAND 

ele_demand = fuel_use_df[fuel_use_df["Parameters"] == "Fuel Consumption"]
ele_demand = ele_demand[ele_demand["Fuel"] == "Electricity"]


ele_demand = (
    ele_demand
    .groupby(standard_group_categories + ["Fuel"])
    .sum("PV")
    .reset_index()
)

ele_demand["PV"] /= 3.6
ele_demand["Unit"] = "TWh"

sns.relplot(
    data = ele_demand, 
    x = "Period", y = "PV", hue = "Scenario",
    kind = "line"    
)

plt.title("Total ele demand")
plt.xlabel("Year")
plt.ylabel("TWh") # not co2e?? 
plt.show()

ele_demand



In [ ]:
# ELECTRICIY DEMAND BY SECTOR

ele_demand = fuel_use_df[fuel_use_df["Parameters"] == "Fuel Consumption"]
ele_demand = ele_demand[ele_demand["Fuel"] == "Electricity"]


ele_demand = (
    ele_demand
    .groupby(standard_group_categories + ["Sector"])
    .sum("PV")
    .reset_index()
)

ele_demand["PV"] /= 3.6
ele_demand["Unit"] = "TWh"

g = sns.relplot(
    data = ele_demand, 
    x = "Period", y = "PV", hue = "Scenario",
    kind = "line",
    col = "Sector", 
    col_wrap = 5,
    facet_kws={'sharey': False}
)
g.figure.suptitle("Electricity Demand by Sector", y = 1.1, size = 20)
g.set_titles(col_template="{col_name}", size = 14)
plt.ylabel("TWh")

g




In [ ]:
# ELECTRICIY DEMAND BY SUBSECTOR

ele_demand = fuel_use_df[fuel_use_df["Parameters"] == "Fuel Consumption"]
ele_demand = ele_demand[ele_demand["Fuel"] == "Electricity"]


ele_demand = (
    ele_demand
    .groupby(standard_group_categories + ["Subsector"])
    .sum("PV")
    .reset_index()
)

ele_demand["PV"] /= 3.6
ele_demand["Unit"] = "TWh"

g = sns.relplot(
    data = ele_demand, 
    x = "Period", y = "PV", hue = "Scenario",
    kind = "line",
    col = "Subsector", 
    col_wrap = 5,
    facet_kws={'sharey': True}
    # facet_kws={'sharey': False}
)
g.figure.suptitle("Electricity Demand by Sector", y = 1.01, size = 20)
g.set_titles(col_template="{col_name}", size = 14)
plt.ylabel("TWh")

g



## SERVICE DEMAND

In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

# Initialize the Dash app
app = dash.Dash(__name__)

def create_facet_chart(df, row_param, col_param):
    # Remove any empty or nan values
    df = df.dropna(subset=[row_param, col_param, 'Period', 'PV'])
    
    # Create a figure with subplots
    fig = px.bar(df,
                 x='Period',
                 y='PV',
                 color='Scenario',
                 facet_row=row_param,
                 facet_col=col_param,
                 barmode='group',
                 height=250 * df[row_param].nunique(),  # Adjust height based on number of rows
                 width=250 * df[col_param].nunique())   # Adjust width based on number of columns
    
    # Update layout for better readability
    fig.update_layout(
        margin=dict(l=50, r=50, t=50, b=50),
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        # Force same y-axis range across all subplots
        yaxis=dict(matches=None)
    )
    
    # Update axes labels and format
    fig.update_xaxes(title_text="Period", tickangle=45)
    fig.update_yaxes(title_text="PV")
    
    # Update subplot titles
    for annotation in fig.layout.annotations:
        annotation.text = annotation.text.split("=")[1]  # Remove the variable name from subplot titles
    
    # Ensure consistent y-axis ranges across facets
    y_max = df['PV'].max() * 1.1  # Add 10% padding
    fig.update_yaxes(range=[0, y_max])
    
    return fig

# Define the layout
app.layout = html.Div([
    html.H1("Energy System Dashboard", 
            style={'textAlign': 'center', 'margin-bottom': '20px'}),
    
    html.Div([
        dcc.Graph(id='facet-chart',
                 config={'displayModeBar': True,
                        'scrollZoom': True,
                        'modeBarButtonsToRemove': ['lasso2d', 'select2d']})
    ], style={'margin': '20px'})
])

# Callback to update the chart
@app.callback(
    Output('facet-chart', 'figure'),
    [Input('facet-chart', 'relayoutData')]
)
def update_chart(relayout_data):
    return create_facet_chart(df, 'Parameters', 'Fuel')

if __name__ == '__main__':
    # Data preprocessing
    # df = df.sort_values(['Parameters', 'Fuel', 'Period'])
    # Remove any duplicates
    # df = df.drop_duplicates(['Parameters', 'Fuel', 'Period', 'Scenario'])
    # Remove rows where Fuel is NaN
    # df = df[df['Fuel'].notna()]
    
    app.run_server(debug=True)

In [ ]:
# ROAD TRANSPORT ENERGY SERVICE DEMAND - TOTAL

# copy above stuff and get a bunch of areas on how this is being filled 

# this is missing biofuel which is a problem. It looks like biofuel was not added correctly.
# surely we should just add biofuel like any other fuel with prices and efficiencies and EFs and everything


vkt_demand = output_df[output_df["Parameters"] == "Distance Travelled"]

vkt_demand = (
    vkt_demand.groupby(standard_group_categories + ["Fuel"])
    .sum("PV")
    .reset_index()
)
vkt_demand

g = sns.FacetGrid(vkt_demand, col='Scenario', aspect = 1.5)  

# Map the plotting function
g.map_dataframe(stacked_area, 
    time_col='Period', 
    value_col='PV', 
    category_col='Fuel'
)

g.figure.suptitle('VKT Demand', y=1.02)
g.add_legend(bbox_to_anchor=(1.05, 1), loc='upper left')
g.set_xlabels("Year")
g.set_ylabels("Billion VKT")


In [ ]:
# ROAD TRANSPORT ENERGY SERVICE DEMAND BY VEHICLE

 
vkt_demand = output_df[output_df["Parameters"] == "Distance Travelled"]

vkt_demand = (
    vkt_demand.groupby(standard_group_categories + ["Fuel", "Enduse"])
    .sum("PV")
    .reset_index()
)

g = sns.FacetGrid(vkt_demand, col='Scenario', row = 'Enduse', aspect = 1.5, sharey=False)  



def stacked_area(data, time_col, value_col, category_col, color=None, **kwargs):
    # Sort the categories to ensure consistent order
    categories = sorted(data[category_col].unique())
    
    pivoted = data.pivot_table(
        index=time_col, 
        columns=category_col, 
        values=value_col,
        aggfunc='sum'
    ).fillna(0)
    
    # Ensure columns are in the same order as categories
    pivoted = pivoted[categories]
    
    # Have to do really weird annoying colour hacks here. There must be an easier way?? 
    colors = {
        'Diesel': '#1e88e5',
        'Electricity': '#43a047',
        'Petrol': '#ff7043',
        'LPG': 'red'
    }
    
    # Use the same order for colors as the data
    color_list = [colors[cat] for cat in categories]
    
    plt.stackplot(
        pivoted.index, 
        pivoted.T,
        labels=categories,  # Use same category order
        colors=color_list,  # Use matched colors
        alpha=0.8, 
    )

# Map the plotting function
g.map_dataframe(stacked_area, 
    time_col='Period', 
    value_col='PV', 
    category_col='Fuel'
)

g.figure.suptitle('Meeting VKT demand', y=1.02)
g.add_legend(bbox_to_anchor=(1.05, 1), loc='upper left')
g.set_xlabels("Year")
g.set_ylabels("Billion VKT")

g.set_titles('{row_name} - {col_name}')


plt.tight_layout()

In [ ]:
# ESD 2: VAR_DEM


output_df

In [ ]:
# ELECTRICITY DEMAND BY TIMESLICE 


# want to display the timeslices we have available
# to do this need to plug into what each timeslice represents 

# Currently setup like this: (24 slices)


"""
Season	Weekly	DayNite

SUM     WK-	      D
FAL     WE-	      N
WIN     	      P
SPR     	

"""

# a couple possibilities for how to represent: 

# average of demand per year, where we split out each bar by the number of hours in it then fill each bar with the elecricity use for that period 
# this should come off of base year but we can take the future years to see what changes 


# peak demand in GW each year (timeseries)
# should try align this with the EA official peak data 

In [ ]:
## PEAK DEMAND 

#1: ele by timeslice 
#2: timeslice size
#3: calculate avg load 
#4: take highest load per year

# is there some way that this is adjusted further? some kind of peak 

# want to also include peak support capacity 


## EVERYTHING BELOW THIS LINE IS BAD AND OLD 

## Agriculture

## energy demand? 

## technology use? 



In [10]:
def make_sector_demand_chart(sector, facet_variable, df = clean_df):
    demand_cols = ["Scenario", "Period", "Sector", "Unit", facet_variable]    

    chart_data = aggregate_demand_data(df, demand_cols)
    chart_data = chart_data[chart_data["Sector"] == sector]

    unit = chart_data["Unit"].unique()[0]

    g = sns.relplot(
        data = chart_data, 
        x = "Period", y = "Value", hue = "Scenario",
        kind = "line",
        col = facet_variable,         
        col_wrap = 4,
        facet_kws={'sharey': False}
        )
    
    g.figure.suptitle(f"{sector} energy demand by {facet_variable}", y = 1.05, size = 20)
    g.set_titles(col_template="{col_name}", size = 14)
    g.set_axis_labels("Year", unit)   


def make_sector_tech_x_fuel_chart(sector, df = clean_df):

    demand_cols = ["Scenario", "Period", "Sector", "Unit", "Technology_Group", "Fuel"]   

    chart_data = aggregate_demand_data(df, demand_cols)
    
    chart_data = chart_data[chart_data["Sector"] == sector]

    unit = chart_data["Unit"].unique()[0]

    g = sns.relplot(
        data = chart_data, 
        x = "Period", y = "Value", hue = "Scenario",
        kind = "line",
        col = "Technology_Group",         
        row = "Fuel",
        # col_wrap = 4,
        facet_kws={'sharey': False}
        )
    
    g.figure.suptitle(f"{sector} energy demand", y = 1.05, size = 20)
    g.set_titles(col_template="{col_name}", size = 14)
    g.set_axis_labels("Year", unit)   
    

def make_sector_demand_charts(sector):
    for variable in ["Fuel", "Subsector", "Technology_Group"]:
        make_sector_demand_chart(sector, variable)
    # make_sector_tech_x_fuel_chart(sector)



want ESD (ie control for the efficiency of fuel used) so we can get the purest form of activity demand

then we would see how that is serviced just to get a better intuition 

for now a better approach might be to get a technology group X fuel split? 